In [107]:
%load_ext autoreload
%autoreload 2
import os
import typing as T
import pickle as pkl
import matplotlib.pyplot as plt
import sys
import cv2
import yaml
import requests
import torch
import scipy
import torch.nn as nn
import numpy as np
from xarray import DataArray
from tqdm.auto import tqdm
import io
from PIL import Image
from bioimageio_chatbot.image_processor import *
import numpy as np
from scipy.spatial.distance import cosine
from random import choice

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [112]:
# # Function to compute pairwise cosine distances
# def compute_pairwise_distances(embedded_db : list):
#     n = len(embedded_db)
#     distances = np.zeros((n, n))
#     for i in range(n):
#         for j in range(n):
#             if i != j:
#                 distances[i, j] = cosine(embedded_db[i][2], embedded_db[j][2])
#             else:
#                 # distances[i, j] = np.inf  # Set the diagonal to infinity
#                 distances[i, j] = 0  # Set the diagonal to infinity
#     return distances

# Function to find the minimum distance to any vector in the representative_db
def min_distance_to_representative(vector : list, representative_db : list):
    min_distance = np.inf
    for _, _, rep_vector in representative_db:
        distance = cosine(vector, rep_vector)
        if distance < min_distance:
            min_distance = distance
    return min_distance

def get_representative_images(input_files : list, verbose : bool = False,
                              grayscale : bool = True, random_seed : int = 2010,
                              threshold_distance : float = 0.25, safety_iter : int = 100):
    image_processor = ImageProcessor()
    embedded_db = []
    input_files_interable = tqdm(input_files) if verbose else input_files
    for input_image_path in input_files_interable:
        fname = os.path.basename(input_image_path)
        input_image = image_processor.read_image(input_image_path)
        input_axes = guess_image_axes(input_image.shape)
        embedded_image = image_processor.embed_image(input_image, input_axes, grayscale = grayscale)
        embedded_db.append((fname, input_image_path, embedded_image))
    np.random.seed(random_seed)
    representative_db = []
    safety_counter = 0
    # Add vectors to representative_db until the condition is met
    while True:
        # random_vector = choice(embedded_db)
        random_idx = np.random.choice(len(embedded_db), replace = False)
        random_vector = embedded_db[random_idx]
        if min_distance_to_representative(random_vector[2], representative_db) > threshold_distance:
            representative_db.append(random_vector)
        # Check if all vectors in embedded_db have been compared
        if all(min_distance_to_representative(vector[2], representative_db) <= threshold_distance for vector in embedded_db):
            break
        safety_counter += 1
        if safety_counter > safety_iter * len(input_files):
            sys.exit('Error - sampling loop iterations exceeded past safety number')
    return([v[1] for v in representative_db])

    # # Compute pairwise distances
    # pairwise_distances = compute_pairwise_distances(embedded_db)


In [115]:
input_files = [os.path.join(cellpose_test_dir, x) for x in os.listdir(cellpose_test_dir) if x.endswith('_img.png')]
representative_images = get_representative_images(input_files, verbose = True)

100%|██████████| 68/68 [00:04<00:00, 14.28it/s]


done


In [116]:
db_path = "../tmp/image_db/"
out_dir = "/Users/gkreder/Downloads/cellpose_test_representative_search"
suffix = "cellpose_test"
fig, axes = test_images(representative_images, out_dir, suffix, db_path, single_output_file=True)

100%|██████████| 25/25 [00:16<00:00,  1.55it/s]
